In [1]:
# 📦 Imports
import pandas as pd
import numpy as np
import joblib
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error
from pathlib import Path
from IPython.display import display

# 📁 Carregar dados com caminho absoluto seguro
ethanol_path = Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/data/[Etanol] Trajeto Casa-Escola-UFRN/trackLog-2023-Feb-13_06-38-49_seg.csv")
gasoline_path = Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/data/[Gasolina] Trajeto Casa-Escola-UFRN/trackLog-2022-Dec-01_06-43-57_qui.csv")

df_ethanol = pd.read_csv(ethanol_path)
df_gasoline = pd.read_csv(gasoline_path)

# ✨ Preparo dos dados
def preparar_dados(df):
    df.columns = df.columns.str.strip()
    col_map = {
        'Speed (OBD)(km/h)': 'Speed(OBD)(km/h)',
        'Latitude ': 'Latitude',
        'Longitude ': 'Longitude'
    }
    df.rename(columns={k: v for k, v in col_map.items() if k in df.columns}, inplace=True)
    if "Speed(OBD)(km/h)" in df.columns:
        speeds = df["Speed(OBD)(km/h)"].fillna(0).values * 1000 / 3600
        df["Acceleration"] = np.diff(speeds, prepend=speeds[0])
    return df

df_ethanol = preparar_dados(df_ethanol)
df_gasoline = preparar_dados(df_gasoline)

# 📦 Carregar modelos
modelos = {
    "ethanol": {
        "afr": joblib.load(Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/LGBMRegressor_ethanol_afr.pkl")),
        "maf": joblib.load(Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/XGBRegressor_ethanol_maf.pkl"))
    },
    "gasoline": {
        "afr": joblib.load(Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/LGBMRegressor_gasoline_afr.pkl")),
        "maf": joblib.load(Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/XGBRegressor_gasoline_maf.pkl"))
    }
}

# 🔧 Função compute_co2
def compute_co2(afr, maf, fuel_type="ethanol"):
    carbon_fraction = {
        "ethanol": 0.5217,
        "gasoline": 0.8571
    }
    c_fraction = carbon_fraction[fuel_type]
    fuel_rate = maf / afr
    return fuel_rate * c_fraction * (44.0 / 12.0)

# 🔧 Função auxiliar para encontrar colunas reais de AFR e MAF
def encontrar_coluna(df, alternativas):
    df.columns = df.columns.str.strip()
    for alt in alternativas:
        if alt in df.columns:
            return df[alt]
    return None

# 🔍 Função principal para executar os testes com visualização Plotly e cálculo de métricas
def executar_teste_plotly(df_original, df_modificado, modelo_afr, modelo_maf, fuel, nome_mr):
    features = ["Latitude", "Longitude", "Speed(OBD)(km/h)", "Acceleration"]

    afr_real = encontrar_coluna(df_original, ["Air Fuel Ratio(Commanded)(:1)", "AFR", "AirFuelRatio(Commanded)(:1)"])
    maf_real = encontrar_coluna(df_original, ["Mass Air Flow Rate(g/s)", "MAF", "MassAirFlowRate(g/s)"])

    co2_real = compute_co2(afr_real.values, maf_real.values, fuel) if afr_real is not None and maf_real is not None else None

    afr_orig = modelo_afr.predict(df_original[features])
    maf_orig = modelo_maf.predict(df_original[features])
    co2_orig = compute_co2(afr_orig, maf_orig, fuel)

    afr_mod = modelo_afr.predict(df_modificado[features])
    maf_mod = modelo_maf.predict(df_modificado[features])
    co2_mod = compute_co2(afr_mod, maf_mod, fuel)

    # 📏 Cálculo de métricas
    diff = np.abs(co2_orig - co2_mod)
    media = diff.mean()
    maxima = diff.max()
    rmse = mean_squared_error(co2_orig, co2_mod, squared=False)
    std = diff.std()

    print(f"\n🔁 {nome_mr} ({fuel.upper()})")
    print(f"Diferença Média: {media:.4f} | Diferença Máxima: {maxima:.4f} | Erro Quadrático Médio (RMSE): {rmse:.4f} | Desvio Padrão (Std): {std:.4f}")

    # 🧾 Tabela comparativa original vs modificado com separação visual e CO₂s
    tabela = pd.concat([
        df_original[features].iloc[:10].add_suffix("_Orig"),
        pd.DataFrame({" ": ["→"] * 10}),
        df_modificado[features].iloc[:10].add_suffix("_Mod"),
        pd.DataFrame({" ": [" "] * 10}),
        pd.DataFrame({
            "CO₂_Real": co2_real[:10] if co2_real is not None else np.nan,
            "CO₂_Orig": co2_orig[:10],
            "CO₂_Mod": co2_mod[:10]
        })
    ], axis=1)
    display(tabela.style.set_caption(f"Comparação dos dados originais e modificados - {nome_mr} ({fuel.upper()})"))

    # 📊 Gráfico Plotly
    fig = go.Figure()
    if co2_real is not None:
        fig.add_trace(go.Scatter(y=co2_real[:100], mode='lines', name='Real', line=dict(dash='dot', color='green')))
    fig.add_trace(go.Scatter(y=co2_orig[:100], mode='lines', name='Predição Original', line=dict(dash='dash', color='blue')))
    fig.add_trace(go.Scatter(y=co2_mod[:100], mode='lines', name='Predição com RM', line=dict(color='red')))

    fig.update_layout(
        title=f"Comparação CO₂ - {nome_mr} - {fuel.upper()}",
        xaxis_title="Amostras",
        yaxis_title="CO₂ (g/s)"
    )
    fig.show()
 

# 🔁 Testes Metamórficos
relacoes_metamorficas = {
    "RM1 - Velocidade constante": lambda df: df.assign(**{"Speed(OBD)(km/h)": df["Speed(OBD)(km/h)"].mean()}),
    "RM2 - Latitude e Longitude constantes": lambda df: df.assign(
        Latitude=df["Latitude"].mean(), Longitude=df["Longitude"].mean()),
    "RM3 - Aceleração constante": lambda df: df.assign(Acceleration=df["Acceleration"].mean())
}

# 🧲 Executar para ambos os combustíveis e RMs
for nome_mr, transformacao in relacoes_metamorficas.items():
    for combustivel, df_original in [("ethanol", df_ethanol), ("gasoline", df_gasoline)]:
        df_modificado = transformacao(df_original.copy())
        executar_teste_plotly(
            df_original=df_original,
            df_modificado=df_modificado,
            modelo_afr=modelos[combustivel]["afr"],
            modelo_maf=modelos[combustivel]["maf"],
            fuel=combustivel,
            nome_mr=nome_mr
        )


🔁 RM1 - Velocidade constante (ETHANOL)
Diferença Média: 0.3081 | Diferença Máxima: 1.7321 | Erro Quadrático Médio (RMSE): 0.4077 | Desvio Padrão (Std): 0.2670


,Latitude_Orig,Longitude_Orig,Speed(OBD)(km/h)_Orig,Acceleration_Orig,,Latitude_Mod,Longitude_Mod,Speed(OBD)(km/h)_Mod,Acceleration_Mod,,CO₂_Real,CO₂_Orig,CO₂_Mod
0,-5.820105,-35.220818,3,0.000000,→,-5.820105,-35.220818,20.896737,0.000000,,0.642839,0.742631,1.447286
1,-5.820103,-35.220828,5,0.555556,→,-5.820103,-35.220828,20.896737,0.555556,,0.653249,0.771505,1.459546
2,-5.820102,-35.220838,5,0.000000,→,-5.820102,-35.220838,20.896737,0.000000,,0.653249,0.771505,1.447286
3,-5.820096,-35.220843,3,-0.555556,→,-5.820096,-35.220843,20.896737,-0.555556,,0.679275,0.659013,0.666306
4,-5.820091,-35.220850,0,-0.833333,→,-5.820091,-35.220850,20.896737,-0.833333,,0.610306,0.667689,0.853810
5,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,20.896737,0.000000,,0.610306,0.706858,1.556241
6,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,20.896737,0.000000,,0.490587,0.706858,1.556241
7,-5.820085,-35.220856,3,0.833333,→,-5.820085,-35.220856,20.896737,0.833333,,1.090483,1.137803,1.569425
8,-5.820074,-35.220859,3,0.000000,→,-5.820074,-35.220859,20.896737,0.000000,,1.090483,1.137803,1.556241
9,-5.820064,-35.220858,3,0.000000,→,-5.820064,-35.220858,20.896737,0.000000,,0.921315,1.137803,1.556241



🔁 RM1 - Velocidade constante (GASOLINE)
Diferença Média: 0.3768 | Diferença Máxima: 1.3891 | Erro Quadrático Médio (RMSE): 0.5071 | Desvio Padrão (Std): 0.3394


,Latitude_Orig,Longitude_Orig,Speed(OBD)(km/h)_Orig,Acceleration_Orig,,Latitude_Mod,Longitude_Mod,Speed(OBD)(km/h)_Mod,Acceleration_Mod,,CO₂_Real,CO₂_Orig,CO₂_Mod
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,-35.220825,24.494192,0.000000,,1.002671,1.456781,2.005537
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,-35.220822,24.494192,-0.277778,,1.013360,1.016708,1.239649
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,-35.220819,24.494192,0.277778,,2.088720,1.456781,2.011935
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,-35.220825,24.494192,0.555556,,2.261889,1.996380,2.011935
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,-35.220829,24.494192,0.555556,,2.499195,2.400486,1.903163
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,-35.220824,24.494192,0.833333,,2.606089,2.400486,1.903163
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,-35.220823,24.494192,0.833333,,3.070012,2.148721,1.831330
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,-35.220810,24.494192,0.833333,,2.445747,1.470940,1.152140
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,-35.220794,24.494192,0.000000,,1.073221,1.588029,1.148476
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,-35.220775,24.494192,-0.555556,,0.562265,1.085921,1.103385



🔁 RM2 - Latitude e Longitude constantes (ETHANOL)
Diferença Média: 0.3162 | Diferença Máxima: 2.3747 | Erro Quadrático Médio (RMSE): 0.5145 | Desvio Padrão (Std): 0.4059


...

,Latitude_Orig,Longitude_Orig,Speed(OBD)(km/h)_Orig,Acceleration_Orig,,Latitude_Mod,Longitude_Mod,Speed(OBD)(km/h)_Mod,Acceleration_Mod,,CO₂_Real,CO₂_Orig,CO₂_Mod
0,-5.820010,-35.220825,5,0.000000,→,-5.812398,-35.208822,5,0.000000,,1.002671,1.456781,1.240028
1,-5.820013,-35.220822,4,-0.277778,→,-5.812398,-35.208822,4,-0.277778,,1.013360,1.016708,0.967113
2,-5.820016,-35.220819,5,0.277778,→,-5.812398,-35.208822,5,0.277778,,2.088720,1.456781,1.240028
3,-5.820014,-35.220825,7,0.555556,→,-5.812398,-35.208822,7,0.555556,,2.261889,1.996380,1.515614
4,-5.819992,-35.220829,9,0.555556,→,-5.812398,-35.208822,9,0.555556,,2.499195,2.400486,1.704046
5,-5.819974,-35.220824,12,0.833333,→,-5.812398,-35.208822,12,0.833333,,2.606089,2.400486,1.704046
6,-5.819944,-35.220823,15,0.833333,→,-5.812398,-35.208822,15,0.833333,,3.070012,2.148721,1.800354
7,-5.819908,-35.220810,18,0.833333,→,-5.812398,-35.208822,18,0.833333,,2.445747,1.470940,1.841241
8,-5.819866,-35.220794,18,0.000000,→,-5.812398,-35.208822,18,0.000000,,1.073221,1.588029,1.826487
9,-5.819821,-35.220775,16,-0.555556,→,-5.812398,-35.208822,16,-0.555556,,0.562265,1.085921,1.011901



🔁 RM3 - Aceleração constante (ETHANOL)
Diferença Média: 0.2013 | Diferença Máxima: 1.8051 | Erro Quadrático Médio (RMSE): 0.3412 | Desvio Padrão (Std): 0.2756


,Latitude_Orig,Longitude_Orig,Speed(OBD)(km/h)_Orig,Acceleration_Orig,,Latitude_Mod,Longitude_Mod,Speed(OBD)(km/h)_Mod,Acceleration_Mod,,CO₂_Real,CO₂_Orig,CO₂_Mod
0,-5.820105,-35.220818,3,0.000000,→,-5.820105,-35.220818,3,-0.000000,,0.642839,0.742631,0.659013
1,-5.820103,-35.220828,5,0.555556,→,-5.820103,-35.220828,5,-0.000000,,0.653249,0.771505,0.830598
2,-5.820102,-35.220838,5,0.000000,→,-5.820102,-35.220838,5,-0.000000,,0.653249,0.771505,0.830598
3,-5.820096,-35.220843,3,-0.555556,→,-5.820096,-35.220843,3,-0.000000,,0.679275,0.659013,0.659013
4,-5.820091,-35.220850,0,-0.833333,→,-5.820091,-35.220850,0,-0.000000,,0.610306,0.667689,0.667689
5,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,0,-0.000000,,0.610306,0.706858,0.667689
6,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,0,-0.000000,,0.490587,0.706858,0.667689
7,-5.820085,-35.220856,3,0.833333,→,-5.820085,-35.220856,3,-0.000000,,1.090483,1.137803,0.724289
8,-5.820074,-35.220859,3,0.000000,→,-5.820074,-35.220859,3,-0.000000,,1.090483,1.137803,0.724289
9,-5.820064,-35.220858,3,0.000000,→,-5.820064,-35.220858,3,-0.000000,,0.921315,1.137803,0.724289



🔁 RM3 - Aceleração constante (GASOLINE)
Diferença Média: 0.3311 | Diferença Máxima: 2.1784 | Erro Quadrático Médio (RMSE): 0.4981 | Desvio Padrão (Std): 0.3721


,Latitude_Orig,Longitude_Orig,Speed(OBD)(km/h)_Orig,Acceleration_Orig,,Latitude_Mod,Longitude_Mod,Speed(OBD)(km/h)_Mod,Acceleration_Mod,,CO₂_Real,CO₂_Orig,CO₂_Mod
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,-35.220825,5,-0.000733,,1.002671,1.456781,1.016708
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,-35.220822,4,-0.000733,,1.013360,1.016708,1.016708
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,-35.220819,5,-0.000733,,2.088720,1.456781,1.016708
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,-35.220825,7,-0.000733,,2.261889,1.996380,0.972466
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,-35.220829,9,-0.000733,,2.499195,2.400486,1.621746
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,-35.220824,12,-0.000733,,2.606089,2.400486,1.630442
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,-35.220823,15,-0.000733,,3.070012,2.148721,1.361783
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,-35.220810,18,-0.000733,,2.445747,1.470940,0.924451
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,-35.220794,18,-0.000733,,1.073221,1.588029,1.040891
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,-35.220775,16,-0.000733,,0.562265,1.085921,1.085921
